In [1]:
import os
from os import listdir
from xml.etree import ElementTree
import tkinter as tk
import shutil as sh
from PIL import Image, ImageTk, ImageEnhance

In [4]:
class Library():
    def __init__(self,dir_img_mask = 'artefacts/mask' , dir_img_orig = 'artefacts/images', 
                 dir_annot = 'artefacts/annotation', dir_save_mask = 'artefacts/library', 
                 dir_save_orig = 'artefacts/library_original'):
        self.dir_img_mask = dir_img_mask
        self.dir_img_orig = dir_img_orig
        self.dir_annot = dir_annot
        self.dir_save_mask = dir_save_mask
        self.dir_save_orig = dir_save_orig
        
    def extract_boxes(self, annot_file):
        # load and parse the file
        tree = ElementTree.parse(annot_file)
        # get the root of the document
        root = tree.getroot()
        # extract each bounding box
        boxes = list()
        for box in root.findall('.//bndbox'):
            xmin = int(box.find('xmin').text)
            ymin = int(box.find('ymin').text)
            xmax = int(box.find('xmax').text)
            ymax = int(box.find('ymax').text)
            coors = [xmin, ymin, xmax, ymax]
            boxes.append(coors)
        # extract image dimensions
        width = int(root.find('.//size/width').text)
        height = int(root.find('.//size/height').text)
        return boxes, width, height
    
    
    def resize_img(self, artefact):
        return artefact.resize((100,100))
#     resize the image preserving the aspect ratio
#         basewidth = 100
#         wpercent = (basewidth/float(artefact.size[0]))
#         hsize = int((float(artefact.size[1])*float(wpercent)))
#         return artefact.resize((basewidth,hsize), Image.ANTIALIAS)
        
    def create_library(self, mask = True):
        if mask==True:
            dir_img = self.dir_img_mask
            dir_save = self.dir_save_mask
        else:
            dir_img = self.dir_img_orig
            dir_save = self.dir_save_orig
            
        # Loop through all the images in the directory dir_img
        for image in listdir(self.dir_img_mask):
            img = Image.open(dir_img + '/' + image)
            name = image.split('.')[0]
            boxes, width, height = self.extract_boxes(self.dir_annot + '/' + name + '.xml')
            # Loop through all the boxes marked in the image
            for index, box in enumerate(boxes,1):
        #       print(name,index, box, width, height)
                artefact = img.crop(box)
                artefact = self.resize_img(artefact)
                # Save the artefact and its index number for any image
                artefact.save(dir_save + '/' + name + '__' + str(index) + '.png')
                
                     
    def classify_one_in_lib(self,image):
        im = Image.open(self.dir_save_mask + '/' + image)
        im_o = Image.open(self.dir_save_orig + '/' + image)
        im_nm = image.split('.')[0]

        # Move an image to any category (Thunked expression)
        def move_2_dir(dir_nm):
            def fxn():
                if not os.path.exists('artefacts/'+ dir_nm):
                    os.mkdir('artefacts/' + dir_nm)

                sh.move(os.path.join(self.dir_save_orig,image),os.path.join('artefacts/' + dir_nm, image))
                root.destroy()
            return fxn

        # Initialising Widget
        root = tk.Tk()

        # For Full Screen GUI
        root.attributes('-fullscreen', True)
        
        # To exit Full Screen
        root.bind("<Escape>", lambda event: root.attributes("-fullscreen", False))

        # Creating ImageTK object
        img =  ImageTk.PhotoImage(image= im, master = root)
        img_o = ImageTk.PhotoImage(image= im_o, master = root)

        # Creating Buttons
        a = tk.Button(root, text="1-Spots", command=move_2_dir('1_spot'),width = 64, bg = 'green')
        b = tk.Button(root, text="2-Trails", command=move_2_dir('2_trail'),width = 64, bg = 'red')
        c = tk.Button(root, text="3-Thick lines", command=move_2_dir('3_thick_lines'),width = 64, bg = 'green')
        d = tk.Button(root, text="4-Spallation", command=move_2_dir('4_spallation'),width = 64, bg = 'red')
        e = tk.Button(root, text="5-Discontinuous", command=move_2_dir('5_discontinuous'),width = 64, bg = 'green')
        f = tk.Button(root, text="6-Multiple", command=move_2_dir('6_multiple'),width = 64, bg = 'red')
        g = tk.Button(root, text="7-Unknown", command=move_2_dir('7_unknown'),width = 64, bg = 'green')
        h = tk.Button(root, text="8", command=move_2_dir('8'),width = 64, bg = 'red')

        # Display Canvas
        canvas = tk.Canvas(root,width=500,height=500)
        canvas.pack()
        canvas.create_image(100,100, anchor="nw", image=img)
        canvas.create_image(300,100, anchor="nw", image=img_o)
        a.pack()
        b.pack()
        c.pack()
        d.pack()
        e.pack()
        f.pack()
        g.pack()
        h.pack()
        root.mainloop()   
    
    # categorize all images in the directory dir_nm
    def classify_all_in_lib(self,dir_nm):
        for image in os.listdir(dir_nm):
            self.classify_one_in_lib(image)
            
    def save_mask_categories(self,dir_nm):
        if not os.path.exists('artefacts/'+ dir_nm + 'm'):
                    os.mkdir('artefacts/' + dir_nm + 'm')
        for image in listdir('artefacts/'+ dir_nm):
            sh.copyfile(os.path.join(self.dir_save_orig,image),os.path.join('artefacts/' + dir_nm + 'm', image))
            

In [83]:
l = Library()
l.create_library

# Library of masked artefacts
l.create_library(mask= True)

# Library of original artefacts
l.create_library()

# Classify artefacts into categories (GUI)
l.classify_all_in_lib('artefacts/library')

# Save Masks of these categorized artefacts
# Creates a directory 1_m for an input of 1
l.save_mask_categories('1')

In [5]:
# Re-classify an artefact in any category
def re_classify(artefact):
    im = Image.open('artefacts/library/'+ artefact)
    im_o = Image.open('artefacts/library_original/'+ artefact)
    im_nm = artefact.split('.')[0]

    # Move an image to any directory
    def move_2_dir(new_category):
        def fxn():
            if not os.path.exists('artefacts/'+ new_category):
                os.mkdir('artefacts/' + new_category)

            sh.move(os.path.join('artefacts/'+ category, image),os.path.join('artefacts/' + new_category, image))
            root.destroy()
        return fxn

    # Initialising Widget
    root = tk.Tk()

    # For Full Screen GUI
    root.attributes('-fullscreen', True)
    root.bind("<Escape>", lambda event: root.attributes("-fullscreen", False))

    # Creating ImageTK object
    img =  ImageTk.PhotoImage(image= im, master = root)
    img_o = ImageTk.PhotoImage(image= im_o, master = root)

    a = tk.Button(root, text="1- Spots",command=move_2_dir('1'),width = 64, bg = 'green')
    b = tk.Button(root, text="2- Thin Trail", command=move_2_dir('2'),width = 64, bg = 'red')
    c = tk.Button(root, text="3- Thick Trail", command=move_2_dir('3'),width = 64, bg = 'green')
    d = tk.Button(root, text="4- Spallation", command=move_2_dir('4'),width = 64, bg = 'red')
    e = tk.Button(root, text="5- Discontinuous", command=move_2_dir('5'),width = 64, bg = 'green')
    f = tk.Button(root, text="6- Multiple", command=move_2_dir('6'),width = 64, bg = 'red')
    g = tk.Button(root, text="7- Unknown", command=move_2_dir('7'),width = 64, bg = 'green')
    h = tk.Button(root, text="8- Fat electrons", command=move_2_dir('8'),width = 64, bg = 'red')

    canvas = tk.Canvas(root,width=500,height=500)
    canvas.pack()
    canvas.create_image(100,100, anchor="nw", image=img)
    canvas.create_image(300,100, anchor="nw", image=img_o)
    a.pack()
    b.pack()
    c.pack()
    d.pack()
    e.pack()
    f.pack()
    g.pack()
    h.pack()
    root.mainloop()   

In [ ]:
# Re-categorize the unknown category
for artefact in listdir('artefacts/7'):
    re_classify(artefact)

In [6]:
def move_images(dir_to, dir_from, dir_ind):
    if not os.path.exists('artefacts/'+ dir_to):
                    os.mkdir('artefacts/' + dir_to)
    for image in listdir('artefacts/'+ dir_ind):
        sh.copy(os.path.join('artefacts/'+ dir_from,image),os.path.join('artefacts/' + dir_to, image))

In [24]:
move_images('1','library_original','1_spot')

move_images('2','library_original','2_trail')

move_images('3','library_original','3_thick_lines')

move_images('4','library_original','4_spallation')

move_images('5','library_original','5_discontinuous')

move_images('6','library_original','6_multiple')

move_images('7','library_original','7_unknown')

In [7]:
move_images('images','images_all','mask')

In [64]:
def resize_n_save(dir_all, dir_save):
    if not os.path.exists('artefacts/'+ dir_save):
                    os.mkdir('artefacts/' + dir_save)
    for image in listdir('artefacts/'+ dir_all):
        img = Image.open('artefacts/'+ dir_all + '/' + image)
        img = img.resize((50,50))
        img.save('artefacts/' + dir_save + '/' + image)
        

In [54]:
resize_n_save('all', 'all_resized')

In [4]:
# A function to modify annotation file using the library of artefacts into different categories 
def modify_annotation_file(dir_annot, category, ctg_name):
    for artefact in listdir('artefacts/'+ str(category)):
        arr = artefact.split('__')
        name = arr[0]
        number = int(arr[1].split('.')[0]) - 1
        tree = ElementTree.parse('artefacts/' + dir_annot + '/' + name + '.xml')
        root = tree.getroot()
        label = root.findall('object')[number]
        label.find('name').text = ctg_name
        tree.write('artefacts/' + dir_annot + '/' + name + '.xml')
        

In [11]:
tree = ElementTree.parse('artefacts/annotation/EN0215720407M.xml')
root = tree.getroot()
classes = list()
for category in root.findall('.//name'):
  classes.append(category.text)

In [12]:
classes

['Thin_trail',
 'Thin_trail',
 'Thick_trail',
 'Thick_trail',
 'Unknown',
 'Thick_trail',
 'Thick_trail',
 'Unknown',
 'Thick_trail',
 'Fat_electron',
 'Unknown']

In [55]:
modify_annotation_file('annotation',1,'Spot')

modify_annotation_file('annotation',2,'Thin_trail')

modify_annotation_file('annotation',3,'Thick_trail')

modify_annotation_file('annotation',4,'Spallation')

modify_annotation_file('annotation',5,'Unknown')

modify_annotation_file('annotation',6,'Unknown')

modify_annotation_file('annotation',7,'Unknown')

modify_annotation_file('annotation',8,'Fat_electron')

In [13]:
# A function to modify annotation file to background foreground class 
def modify_annotation_file():
    for name in listdir('dataset_70_images/annotation-binary'):
        tree = ElementTree.parse('dataset_70_images/annotation-binary/'+ name)
        root = tree.getroot()
        for label in root.findall('object'):
            label.find('name').text = 'Positive'
            tree.write('dataset_70_images/annotation-binary/'+ name)

In [14]:
modify_annotation_file()

In [5]:
def check_images(dir_negative, dir_ind):
    for image in listdir('/'+ dir_ind):
        if os.path.exists('artefacts/'+ dir_negative + '/' + image):
            os.remove('artefacts/'+ dir_ind + '/' + image)

check_images('Minus','2011_214')